Task-1

8-Puzzle

In [14]:
from queue import PriorityQueue

goal_state = [[1, 2, 3],
              [4, 5, 6],
              [7, 8, 0]]

R = 3
C = 3

initial_state = []
print("Enter the entries rowwise:")
for i in range(R):
    a = []
    for j in range(C):
        a.append(int(input()))
    initial_state.append(a)
print("\n")

def manhattan_distance(state):
    distance = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                row = (state[i][j] - 1) // 3
                col = (state[i][j] - 1) % 3
                distance += abs(i - row) + abs(j - col)
    return distance

class PuzzleNode:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = parent.cost + 1 if parent else 0
        self.heuristic = manhattan_distance(state)
        self.priority = self.cost + self.heuristic

    def __lt__(self, other):
        return self.priority < other.priority

def solve_puzzle(initial_state, goal_state):
    open_set = PriorityQueue()
    closed_set = set()
    initial_node = PuzzleNode(initial_state)
    open_set.put(initial_node)
    while not open_set.empty():
        current_node = open_set.get()
        if current_node.state == goal_state:
            solution_path = []
            while current_node:
                solution_path.append(current_node.state)
                current_node = current_node.parent
            return reversed(solution_path)

        closed_set.add(tuple(map(tuple, current_node.state)))
        row, col = next((i, j) for i in range(3) for j in range(3) if current_node.state[i][j] == 0)
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        for dr, dc in moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_state = [row[:] for row in current_node.state]
                new_state[row][col], new_state[new_row][new_col] = new_state[new_row][new_col], new_state[row][col]
                if tuple(map(tuple, new_state)) not in closed_set:
                    open_set.put(PuzzleNode(new_state, current_node, (dr, dc)))
    return None

solution_path = solve_puzzle(initial_state, goal_state)
if solution_path:
    for state in solution_path:
        for row in state:
            print(row)
        print()
    print("Solved!")
else:
    print("No solution found.")


Enter the entries rowwise:
1
2
3
4
5
6
7
8
0


[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Solved!


Vaccum Cleaner

In [6]:
def clean(floor):
  i, j, row, col = 0, 0, len(floor), len(floor[0])
  for i in range(row):
      if(i%2 == 0):
          for j in range(col):
              if(floor[i][j] == 1):
                  print_F(floor, i, j)
                  floor[i][j] = 0
              print_F(floor, i, j)
      else:
          for j in range(col-1, -1, -1):
              if(floor[i][j] == 1):
                  print_F(floor, i, j)
                  floor[i][j] = 0
              print_F(floor, i, j)
def print_F(floor, row, col):
    """A function to print the GRID , (row, col) represent the current vacuum cleaner position"""
    print("The Floor matrix is as below:")
    for r in range(len(floor)):
        for c in range(len(floor[r])):
            if r == row and c == col:
                print(f" >{floor[r][c]}< ", end = '')
            else:
                print(f"  {floor[r][c]}  ", end = '')
        print(end = '\n')
    print(end = '\n')
def main():
    floor = []
    m = int(input("Enter the No. of Rows: "))
    n = int(input("Enter the No. of Columns: "))
    print("Enter clean status for each cell (1 - dirty, 0 - clean)")
    for i in range(m):
        f = list(map(int, input().split(" ")))
        floor.append(f)
    print()
    clean(floor)
main()

Enter the No. of Rows: 2
Enter the No. of Columns: 2
Enter clean status for each cell (1 - dirty, 0 - clean)
1 0
1 0

The Floor matrix is as below:
 >1<   0  
  1    0  

The Floor matrix is as below:
 >0<   0  
  1    0  

The Floor matrix is as below:
  0   >0< 
  1    0  

The Floor matrix is as below:
  0    0  
  1   >0< 

The Floor matrix is as below:
  0    0  
 >1<   0  

The Floor matrix is as below:
  0    0  
 >0<   0  



Wumpus World

In [9]:
import random

class WumpusWorld:
    def __init__(self, size=4):
        self.size = size
        self.grid = [[[] for _ in range(size)] for _ in range(size)]
        self.agent_position = (0, 0)
        self.agent_alive = True
        self.wumpus_alive = True
        self.gold_position = None
        self.wumpus_position = None
        self.pit_positions = set()

    def initialize(self):
        self.agent_position = (0, 0)
        self.agent_alive = True
        self.wumpus_alive = True
        self.gold_position = self.random_position()
        self.wumpus_position = self.random_position(exclude=[self.gold_position])
        self.pit_positions = set()
        while len(self.pit_positions) < self.size:
            self.pit_positions.add(self.random_position(exclude=[self.gold_position, self.wumpus_position]))

    def random_position(self, exclude=None):
        if exclude is None:
            exclude = []
        while True:
            x = random.randint(0, self.size - 1)
            y = random.randint(0, self.size - 1)
            if (x, y) not in exclude:
                return x, y

    def adjacent_positions(self, position):
        x, y = position
        candidates = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
        return [(nx, ny) for nx, ny in candidates if 0 <= nx < self.size and 0 <= ny < self.size]

    def percept(self):
        x, y = self.agent_position
        perceptions = []
        if (x, y) == self.wumpus_position and self.wumpus_alive:
            perceptions.append("Stench")
        if (x, y) in self.pit_positions:
            perceptions.append("Breeze")
        if (x, y) == self.gold_position:
            perceptions.append("Glitter")
        return perceptions

    def shoot_arrow(self):
        if not self.agent_alive or not self.wumpus_alive:
            return False
        ax, ay = self.agent_position
        wx, wy = self.wumpus_position
        if ax == wx or ay == wy:
            self.wumpus_alive = False
            return True
        return False

    def move(self, direction):
        if not self.agent_alive:
            return False, "Agent is dead."
        ax, ay = self.agent_position
        if direction == "Up":
            nx, ny = ax - 1, ay
        elif direction == "Down":
            nx, ny = ax + 1, ay
        elif direction == "Left":
            nx, ny = ax, ay - 1
        elif direction == "Right":
            nx, ny = ax, ay + 1
        else:
            return False, "Invalid direction."

        if not (0 <= nx < self.size and 0 <= ny < self.size):
            return False, "Out of bounds."

        self.agent_position = (nx, ny)

        if (nx, ny) == self.wumpus_position and self.wumpus_alive:
            self.agent_alive = False
            return False, "Agent is eaten by the Wumpus."

        if (nx, ny) in self.pit_positions:
            self.agent_alive = False
            return False, "Agent falls into a pit."

        return True, "Moved successfully."

    def grab_gold(self):
        if self.agent_position == self.gold_position:
            return True
        return False

    def climb_out(self):
        return self.agent_position == (0, 0) and not self.agent_alive

# Example of usage:

if __name__ == "__main__":
    size = 4
    world = WumpusWorld(size)
    world.initialize()

    print("Initial state:")
    print(f"Agent position: {world.agent_position}")
    print(f"Gold position: {world.gold_position}")
    print(f"Wumpus position: {world.wumpus_position}")
    print(f"Pit positions: {world.pit_positions}")

    print("\nStarting the game:")

    for i in range(size * size):  # Limited steps for demonstration
        print(f"\nTurn {i + 1}:")
        print(f"Agent percept: {world.percept()}")

        action = random.choice(["Up", "Down", "Left", "Right"])
        success, message = world.move(action)
        print(f"Action: {action}")
        print(f"Result: {message}")
        print(f"Agent position: {world.agent_position}")

        if world.agent_alive and world.agent_position == world.gold_position:
            print("Agent found the gold!")
            grabbed = world.grab_gold()
            print(f"Grabbed the gold: {grabbed}")

        if not world.agent_alive:
            print("Agent is dead.")
            break

        if world.agent_position == (0, 0):
            climbed_out = world.climb_out()
            print("Agent climbed out of the cave.")
            print(f"Climbed out successfully: {climbed_out}")
            break

    print("\nFinal state:")
    print(f"Agent position: {world.agent_position}")
    print(f"Agent alive: {world.agent_alive}")
    print(f"Wumpus alive: {world.wumpus_alive}")


Initial state:
Agent position: (0, 0)
Gold position: (1, 1)
Wumpus position: (1, 3)
Pit positions: {(3, 1), (1, 0), (1, 2), (2, 0)}

Starting the game:

Turn 1:
Agent percept: []
Action: Up
Result: Out of bounds.
Agent position: (0, 0)
Agent climbed out of the cave.
Climbed out successfully: False

Final state:
Agent position: (0, 0)
Agent alive: True
Wumpus alive: True
